# NDEx Client Tutorial v3.0


## Overview

The NDEx Python Client is a module that simplifies access to the NDEx Server API and provides convenience methods for common operations on networks.

In this tutorial, we will first show how to install the NDEx Python Client module and use it for basic operations on networks stored on an NDEx server.

The NDEx Python Client provides an interface to an NDEx server that is managed via a client object class.  For each NDEx server and user account that you want to use in your script or application, you create an instance of the NDEx Client. An NDEx Client object can be used to access an NDEx server either anonymously or using a specific user account.


## Requirements

The **NDEx Python Client 3.0** requires Python 2.7.9 and the latest version of the PIP Python package manager for installation.  [Click here](https://pypi.python.org/pypi/pip) to download the PIP Python package.


## Installing the NDEx Python Client Module

The Python Client is available as a development release of the ndex module in the Python Package Index (PyPI) as of February 2017.  This tutorial was composed using the 2.0 release of the ndex module. To install this module to your computer using PIP:
        >>> pip install ndex 
or
        >>> pip install ndex --upgrade 
        
if you have an older version of this module already installed. 


## Setting up NDEx Clients

In this section you will configure two client objects to access the public NDEx server. The first will let you make anonymous requests and the second will let you perform operations requiring authentication, such as saving networks to your account.<br>
With the ndex module installed, start Python and import ndex.client:


In [1]:
import ndex.client as nc
import io
import json

### Anonymous Clients

Create a NDEx client object to access the NDEx public server anonymously:

In [3]:
anon_ndex=nc.Ndex("http://public.ndexbio.org")

### Personal Clients

This kind of clients allows you to perform operations requiring authentication, such as saving networks to your account.<br>
Create a second client object to access the same server with your account. For convenience and clarity in this example, we set the variables ‘my_account’ and ‘my_password’ to hold the strings for your account name and password:


In [2]:
my_account="your username"
my_password="your password"
my_ndex=nc.Ndex("http://public.ndexbio.org",my_account,my_password)

## Working with the NDEx Network Using the Anonymous Client

### Get Network Information by Accession

You can access a network by its accession id, which is an universally unique identifier (UUID) assigned to the network by the NDEx server. All networks have a UUID and they are unique across all servers. No two networks will share an UUID.

In this step, you will get basic information about the network and retrieve a Network Summary structure. The ‘Metabolism of RNA’ network is in the NDEx Tutorials account on the public NDEx server; its UUID is ‘9ed0cd55-9ac0-11e4-9499-000c29202374’



anon_ndex will access the network using the **get_network_summary(network_id)** which returns a NetworkSummary as  
a dictionary:

In [4]:
ns=anon_ndex.get_network_summary('9ed0cd55-9ac0-11e4-9499-000c29202374')
print ns.get('name')
print ns.get('edgeCount')
print ns.get('nodeCount')

Metabolism of RNA
4344
361


### Find Networks by Search – Simple Search


You can search for networks by the text in their name and description as well as the names and controlled vocabulary terms associated with their nodes. The input is a search string that conforms to Lucene search string syntax, but in its simplest form is one or more search terms separated by spaces.

anon_ndex will perform the network search using the **search_networks(search_string="", account_name=None, start=0, size=100, include_groups=False)** method that returns a list of NetworkSummary dictionaries:


In [6]:
metabolic_networks=anon_ndex.search_networks('metabo*')
print len(metabolic_networks['networks'])
for ns in metabolic_networks['networks']: print ns.get('name')

68
humanCyc metabolic Pathways - PC2 v.7 - BioPAX3
Signaling Pathways Subnetwork - 10/10/2014 11:07:44 AM
Metabolism of RNA Subnetwork - 2/13/2015 9:49:09 AM
HumanCyc metabolic pathways
BEL Framework Large Corpus Document
Reconstruction of the human genome
Reconstruction of the human genome - BioPAX3
Kegg Pathways (Human only) - BioPAX3
CTD - Comparative Toxicogenomics Database (Human only) - PC2 v.7
HumanCyc metabolic pathways - PC2 v.7
CTD - Comparative Toxicogenomics Database - XBEL
Prolactin Signalling Pathway
Amine compound SLC transporters
PLC-mediated hydrolysis of PIP2
Steroid hormones
Pathways Affected by Melanoma Genes
RiceCyc v3.3
MaizeCyc v2.0.2
CELL 2015 - Ipilimumab network
CELL 2015 - Ivacaftor network
Signaling Pathways
Transmembrane transport of small molecules
ChEMBL - Medium affinty compounds vs human targets
Metabolism of RNA
TTD - Drug-Disease Associations
BDNF Signaling Pathway
TTD - Target-Disease Associations (Discontinued Targets)
insulin_secretion
ChEMBL - All

The search can also be limited to a specific account and to a number of search results:


In [7]:
metabolic_networks=anon_ndex.search_networks('metabo*', 'ndextutorials', size=2)
print len(metabolic_networks['networks'])
for ns in metabolic_networks['networks']: print ns.get('name')

2
Metabolism
Metabolism of proteins


### Get a Network


You can obtain an entire network as a CX stream, which is an NDEx format that is optimized for streaming networks. 
This is performed as a monolithic operation, so care should be taken when requesting very large networks. Applications
can use the **get_network_summary** method to check the node and edge counts for a network before attempting to use 
**get_complete_network**. The stream is contained in a Response object from the 
[Python requests library](http://docs.python-requests.org/en/master/).


anon_ndex will use the **get_complete_network(network_id)** method to get a Response object (from the [Python requests library](http://docs.python-requests.org/en/master/))

In [15]:
response=anon_ndex.get_network_as_cx_stream('9ed0cd55-9ac0-11e4-9499-000c29202374')
print len(response.text)

665511


### Query a Network – Neighborhood Query



You can retrieve a ‘neighborhood’ subnetwork of a network as CX stream. The query finds the subnetwork by first 
identifying nodes that are associated with identifiers in the search_string, then traversing search_depth number of 
steps from those nodes.  The search_depth is limited to 3 steps and the following example shows a query using the 
default 1-step limit. Once against, the stream is contained in a Response object from the 
[Python requests library](http://docs.python-requests.org/en/master/).


anon_ndex will use the **get_neighborhood_as_cx_stream(network_id, search_string, search_depth=1, edge_limit=2500)** method to get a Response object (from the [Python requests library](http://docs.python-requests.org/en/master/))

In [21]:
response=anon_ndex.get_neighborhood_as_cx_stream('9ed0cd55-9ac0-11e4-9499-000c29202374', 'XRN1')
network=json.loads(response.text)

def getNumberOfNodesAndEdgesFromCX(network):
    numberOfEdges = numberOfNodes = 0;
    for aspect in network:
        if 'metaData' in aspect:
            metaData = aspect['metaData']
            for element in metaData:
                if (('name' in element) and (element['name'] == 'nodes')):
                    numberOfNodes = element['elementCount']
                if (('name' in element) and (element['name'] == 'edges')):
                    numberOfEdges = element['elementCount']
            break
    return numberOfNodes, numberOfEdges


nodes, edges = getNumberOfNodesAndEdgesFromCX(network['data'])

print 'nodes=', nodes, '  edges=', edges



nodes= 20   edges= 26


## Working with the NDEx Network Using Your Personal Client

### Create a Network

You can create a new network on an NDEx server if you have a CX stream. The network is created in the user account associated with the client object. All methods that create or modify content on the NDEx server require authentication, so you will use the my_ndex client object that you set up at the start of the tutorial and will create a network in your account.<br>
An error is returned if the CX stream is larger than a maximum size for network creation set in the NDEx server configuration.


The small network that you obtained based on a query for 26 edges (in the previous section) will be saved to your account in this example. <br>

my_ndex client will use **save_cx_stream_as_new_network(network)** to do that; a URI of the newly-created networkd will be returned to the caller:

In [26]:
cx_stream = json.dumps(network['data'])
uri = my_ndex.save_cx_stream_as_new_network(cx_stream)
print "uri of newly creared network is ", uri

uri of newly creared network is  http://public.ndexbio.org/v2/network/2ba22825-feb2-11e6-aba2-0ac135e8bacf


### Update Network Profile

You can also update just the network profile (defined as the name, description and version of a network) 
on an NDEx server based on the network UUID. You use the method **update_network_profile(network_id, network_profile)**
which does not return anything.

In [3]:
#we use ID of the network we created at previous step; it will be different in your case since network UUIDs are unique
networkId="2ba22825-feb2-11e6-aba2-0ac135e8bacf" 
network_profile={"name":"Renamed Network", "description":"New Description", "version":"2.0"}
my_ndex.update_network_profile(networkId, network_profile)

''

### Set Read-Only

You can also set a network to be read-only using **set_read_only(network_id, boolean)** so that it can be retrieved more quickly by others. The method does not return anything. Please note that a read-only network cannot be edited or 
deleted, so you will need to revert it to its original state  prior to proceeding with the next step in this tutorial. 

In [4]:
networkId="2ba22825-feb2-11e6-aba2-0ac135e8bacf" 
#make network read-only
my_ndex.set_read_only(networkId, True)

''

In [5]:
networkId="2ba22825-feb2-11e6-aba2-0ac135e8bacf" 
#revert network to original state (make it read-write again)
my_ndex.set_read_only(networkId, False)

''

### Delete a Network

Finally, you can delete a network using **delete_network(networkId)** that you own by its UUID. The method does not 
return anything. Be careful: there is no method to undo a deletion.

In [7]:
networkId="2ba22825-feb2-11e6-aba2-0ac135e8bacf" 
my_ndex.delete_network(networkId)

''